# CNN 
Bug fix in selecting y.

In [1]:
DATAPATH=''
try:
    # On Google Drive, set path to my drive / data directory.
    from google.colab import drive
    IN_COLAB = True
    PATH='/content/drive/'
    drive.mount(PATH)
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
except:
    # On home computer, set path to local data directory.
    IN_COLAB = False
    DATAPATH='data/'  # must end in "/"

ZIP_FILE='BuildingData.zip'
ZIP_PATH = DATAPATH+ZIP_FILE
STEAM_FILE='steam.csv'
WEATHER_FILE='weather.csv'
MODEL_FILE='Model'  # will be used later to save models

In [2]:
from os import listdir
import csv
from zipfile import ZipFile
import numpy as np
import pandas as pd
from scipy import stats  # mode

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.losses import MeanSquaredError
from keras.layers import Conv2D
from keras.layers import Flatten

import matplotlib.pyplot as plt
from matplotlib import colors
mycmap = colors.ListedColormap(['red','blue'])  # list color for label 0 then 1
np.set_printoptions(precision=2)

In [3]:
def read_zip_to_panda(zip_filename,csv_filename):
    zip_handle = ZipFile(zip_filename)
    csv_handle = zip_handle.open(csv_filename)
    panda = pd.read_csv(csv_handle)
    return panda
def fix_date_type(panda):
    # Convert the given timestamp column to the pandas datetime data type.
    panda['timestamp'] = pd.to_datetime(panda['timestamp'], infer_datetime_format = True)
    indexed = panda.set_index(['timestamp'])
    return indexed
def get_site_timeseries(panda,site):
    # Assume the panda dataframe has a datetime column.
    # (If not, call fix_date_type() before this.)
    # Extract the timeseries for one site.
    # Convert the datetime column to a DatetimeIndex.
    site_df = panda[panda['site_id']==site]
    temp_col = site_df['date']
    temp_val = temp_col.values
    temp_ndx = pd.DatetimeIndex(temp_val)
    dropped = site_df.drop('date',axis=1)
    panda = dropped.set_index(temp_ndx)
    return panda

## CNN setup

In [4]:
# Before analyzing the entire dataset, we look at this subset.
SITE = 'Eagle'
METER = 'steam'

# Arrange "picture" of weather with temperatures toward the middle
PREDICTED_VARIABLE = 'steam' 
PREDICTORS = ['cloudCoverage', 'airTemperature', 'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']
print("PREDICTORS=",len(PREDICTORS),PREDICTORS)

# Downsample True means collapse 365*24 measures to 365 daily averages
# Downsample False means replace 365*24 measures with 365*24 window averages
DOWNSAMPLE = False   

STEPS_HISTORY = 24   # length of the predictor sequence
STEPS_FUTURE =  1    # length of the predicted sequence

## CNN parameters
EPOCHS=5
FILTERS = 8
WIDTH = 3
STRIDE = (1,1)
INPUT_SHAPE = (STEPS_HISTORY,len(PREDICTORS),1) 

PREDICTORS= 8 ['cloudCoverage', 'airTemperature', 'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']


In [5]:
wet_df = read_zip_to_panda(ZIP_PATH,WEATHER_FILE)
wet_df = fix_date_type(wet_df)
stm_df = read_zip_to_panda(ZIP_PATH,STEAM_FILE)
stm_df = fix_date_type(stm_df)
site_specific_weather = wet_df.loc[wet_df['site_id'] == SITE]
all_buildings = [x for x in stm_df.columns if x.startswith(SITE)]

In [9]:
def smooth(df):
    # For smoothing the 24 hour cycle, we do not want exponential smoothing.
    smoothed = None
    if DOWNSAMPLE:
        # This alternate method samples down to 1/24 time steps.
        smoothed = df.resample("24H").mean() 
    else:
        # This method does not reduce the number of time steps.
        # Note the first 23 measurements get set to Nan.
        smoothed=df.rolling(window=24).mean()
        smoothed=smoothed[24:]
    return smoothed

# Correlation is low when buildings have many NaN and 0 meter readings.
# We will ignore buildings that have >max bad meter readings.
def is_usable_column(df,column_name):
    MAX_BAD = 500 
    bad = df[column_name].isin([0]).sum()
    return bad<=MAX_BAD

def prepare_for_learning(df):
    num_samples = len(df) - STEPS_FUTURE - STEPS_HISTORY
    num_predictors = len(PREDICTORS)
    X_shape = (num_samples,STEPS_HISTORY,num_predictors,1)
    X=np.zeros(X_shape)
    Y_shape = (num_samples,STEPS_FUTURE)
    y=np.zeros(Y_shape)
    predictor_series = df[PREDICTORS].values  # e.g. all weather values
    predicted_series = df[PREDICTED_VARIABLE].values  # e.g. all meter readings
    
    for x0 in range (0,num_samples): # Loop over all 1000 samples
        # This is one array of weather for previous 24 time periods
        one_sample = predictor_series[x0:x0+STEPS_HISTORY]
        one_label =  predicted_series[x0+STEPS_HISTORY:x0+STEPS_FUTURE]
        # Loop over all 24 time periods
        for x1 in range (0,STEPS_HISTORY): # In 1 sample, loop over 24 time periods
            one_period = one_sample[x1]
            for x2 in range (0,num_predictors): # In 1 time period, loop over 8 weather metrics
                one_predictor = one_period[x2]
                # for x3 in range (0,X_shape[3]): # In 1 metric, loop over vector dimensions
                # In our data, each weather metric is a scalar.
                x3 = 0
                X[x0,x1,x2,x3] = one_predictor
        y[x0]=predicted_series[x0:x0+STEPS_FUTURE]
    return X,y 


In [10]:
def make_CNN():
    print("make_CNN")
    print("input shape:",INPUT_SHAPE)
    cnn = Sequential()
    cnn.add(
        Conv2D( input_shape=INPUT_SHAPE,
            filters=FILTERS,kernel_size=WIDTH,strides=STRIDE,
            activation=None, padding="valid"))
    cnn.add(Flatten())
    cnn.add(Dense(STEPS_FUTURE))   
    cnn.compile(optimizer='adam',loss=MeanSquaredError())
    return cnn

## Process all buildings

In [11]:
cors = []
ONE_PREDICTOR = 'dewTemperature'  ## illustrate difficulty by showing best correlate
for BLDG in all_buildings:
    print("Building",BLDG)
    # Get steam usage for one building.
    bldg_specific_steam = stm_df[[BLDG]]
    # Concatenate steam usage with weather.
    one_bldg_df = pd.concat([bldg_specific_steam,site_specific_weather],axis=1)
    # Drop the site, which is constant (we selected for one site).
    one_bldg_df = one_bldg_df.drop(['site_id'],axis=1)
    # The original steam table used column name = building name.
    # We are processing one building, so rename to the column 'steam'.
    one_bldg_df = one_bldg_df.rename(columns={BLDG : METER})
    # In order to filter bad buildings, count sum of NaN + zero.
    one_bldg_df = one_bldg_df.fillna(0)
    
    if is_usable_column(one_bldg_df,METER):
        one_bldg_df = smooth(one_bldg_df) 
        X,y = prepare_for_learning(one_bldg_df)
        # Ideally, split Year1 = train, Year2 = test.
        # Some data is incomplete, so split 1st half and 2nd half.
        split = len(X)//2 
        X_train = np.asarray(X[0:split])
        y_train = np.asarray(y[0:split])
        X_test = np.asarray(X[split:])
        y_test = np.asarray(y[split:])
        print("Train on",len(X_train),"samples such as",X_train[100][0])
        print("Predict",len(y_train),"labels such as",y_train[100])

        model = make_CNN()
        print(model.summary())
        print("X_train.shape:",X_train.shape)
        model.fit(X_train,y_train,epochs=EPOCHS)
        y_pred = model.predict(X_test)
        # Compare. Solve the problem that predict.shape != truth.shape 
        #print(" before ytestshape",y_test.shape,"ypredshape",y_pred.shape)
        #nsamples, nsteps, ndim = y_test.shape
        #y_test = y_test.reshape(nsamples,nsteps*ndim)
        #nsamples, nsteps, ndim = y_pred.shape
        #y_pred = y_pred.reshape(nsamples,nsteps*ndim)
        ##print(" after ytestshape",y_test.shape,"ypredshape",y_pred.shape)
        rmse = mean_squared_error(y_test,y_pred,squared=False)
        # Keep a table for reporting later.
        mean = one_bldg_df[METER].mean()
        cor = one_bldg_df.corr().loc[PREDICTED_VARIABLE][ONE_PREDICTOR] 
        cors.append([cor,mean,rmse,rmse/mean,BLDG])
        print("cor,mean,rmse,rmse/mean,bldg:",cor,mean,rmse,rmse/mean,BLDG)

        ## break   ## REMOVE THIS LINE TO LOOP OVER BUILDINGS!
        
if True:
    print("History",STEPS_HISTORY,"Future",STEPS_FUTURE)
    print("Column 1: Correlation of",PREDICTED_VARIABLE,"and",ONE_PREDICTOR)
    print("          Using one weather feature as leading correlate.")
    print("Column 2: Mean usage.")
    print("          Using mean to help understand the RMSE.")
    print("Column 3: RMSE of LinearRegression(X=Weather, y=Usage).")
    print("Column 4: RMSE/mean normalized to help understand RMSE.")
    print("Column 5: Building.")
    for cor in sorted(cors):
        print("%7.4f %10.2f %10.2f %5.2f   %s"%(cor[0],cor[1],cor[2],cor[3],cor[4]))    

Building Eagle_office_Lamont
Building Eagle_health_Athena
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.65e+02]
  [ 1.57e+00]]

 [[ 0.00e+00]
  [-6.83e+00]
  [-1.66e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.50e+02]
  [ 1.40e+00]]

 [[ 0.00e+00]
  [-6.69e+00]
  [-1.62e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.35e+02]
  [ 1.20e+00]]

 [[ 0.00e+00]
  [-6.48e+00]
  [-1.58e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.32e+02]
  [ 1.16e+00]]

 [[ 0.00e+00]
  [-6.23e+00]
  [-1.54e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.27e+02]
  [ 1.06e+00]]



274/274 [==============================] - 1s 3ms/step - loss: 3590.6381
Epoch 5/5
274/274 [==============================] - 1s 2ms/step - loss: 3522.6895
cor,mean,rmse,rmse/mean,bldg: -0.759617507777322 181.93965687500085 65.90671578398623 0.36224491634205147 Eagle_public_Alvin
Building Eagle_education_Raul
Building Eagle_education_Roman
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.65e+02]
  [ 1.57e+00]]

 [[ 0.00e+00]
  [-6.83e+00]
  [-1.66e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.50e+02]
  [ 1.40e+00]]

 [[ 0.00e+00]
  [-6.69e+00]
  [-1.62e+01]
  [ 0.00e+00]
  

274/274 [==============================] - 2s 5ms/step - loss: 1825.1529
Epoch 2/5
274/274 [==============================] - 2s 6ms/step - loss: 983.0838
Epoch 3/5
274/274 [==============================] - 1s 5ms/step - loss: 990.1121
Epoch 4/5
274/274 [==============================] - 2s 6ms/step - loss: 903.5767
Epoch 5/5
274/274 [==============================] - 2s 6ms/step - loss: 896.3447
cor,mean,rmse,rmse/mean,bldg: -0.7222281821273872 81.87169456264256 31.031665010830267 0.379028004447703 Eagle_lodging_Edgardo
Building Eagle_education_Cassie
Building Eagle_education_Peter
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e

274/274 [==============================] - 2s 4ms/step - loss: 6242.0722
Epoch 2/5
274/274 [==============================] - 1s 5ms/step - loss: 1433.4993
Epoch 3/5
274/274 [==============================] - 1s 4ms/step - loss: 1616.8566
Epoch 4/5
274/274 [==============================] - 1s 5ms/step - loss: 1468.2060
Epoch 5/5
274/274 [==============================] - 1s 5ms/step - loss: 1347.4804
cor,mean,rmse,rmse/mean,bldg: -0.7129673388128998 92.73484781606874 47.99169860587164 0.5175152570591249 Eagle_lodging_Dawn
Building Eagle_office_Nereida
Building Eagle_lodging_Tressa
Building Eagle_education_Eileen
Building Eagle_education_Wesley
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1

274/274 [==============================] - 2s 3ms/step - loss: 10290.3267
Epoch 2/5
274/274 [==============================] - 1s 3ms/step - loss: 617.0520
Epoch 3/5
274/274 [==============================] - 1s 3ms/step - loss: 566.8096
Epoch 4/5
274/274 [==============================] - 1s 3ms/step - loss: 561.1823
Epoch 5/5
274/274 [==============================] - 1s 3ms/step - loss: 625.2676
cor,mean,rmse,rmse/mean,bldg: -0.7961450049576672 121.90840096508735 24.98271127688332 0.20493018593556958 Eagle_health_Vincenza
Building Eagle_office_Dallas
Building Eagle_education_Shante
Building Eagle_office_Chauncey
Building Eagle_office_Phyllis
Building Eagle_office_Freida
Building Eagle_office_Francis
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  

274/274 [==============================] - 2s 5ms/step - loss: 1327522.2418
Epoch 2/5
274/274 [==============================] - 1s 5ms/step - loss: 334758.2884
Epoch 3/5
274/274 [==============================] - 2s 6ms/step - loss: 288808.4686
Epoch 4/5
274/274 [==============================] - 2s 6ms/step - loss: 271363.1445
Epoch 5/5
274/274 [==============================] - 1s 5ms/step - loss: 251458.9439
cor,mean,rmse,rmse/mean,bldg: -0.8876757978269461 2030.3587884358292 451.78841176720425 0.22251653960887283 Eagle_education_Sherrill
Building Eagle_education_Brooke
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e+01]
  [ 0

274/274 [==============================] - 1s 4ms/step - loss: 170594.4467
Epoch 2/5
274/274 [==============================] - 1s 4ms/step - loss: 77269.0152
Epoch 3/5
274/274 [==============================] - 1s 5ms/step - loss: 73270.3886
Epoch 4/5
274/274 [==============================] - 1s 5ms/step - loss: 69820.4882
Epoch 5/5
274/274 [==============================] - 1s 4ms/step - loss: 70069.4924
cor,mean,rmse,rmse/mean,bldg: -0.7722467071075976 711.3309670245889 226.68538034785465 0.31867778974399513 Eagle_education_Norah
Building Eagle_education_Will
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e+01]
  [ 0.00e+00]
  

274/274 [==============================] - 2s 4ms/step - loss: 4873.0564
Epoch 2/5
274/274 [==============================] - 1s 4ms/step - loss: 438.6233
Epoch 3/5
274/274 [==============================] - 1s 5ms/step - loss: 408.9482
Epoch 4/5
274/274 [==============================] - 1s 5ms/step - loss: 422.1827
Epoch 5/5
274/274 [==============================] - 1s 5ms/step - loss: 390.8835
cor,mean,rmse,rmse/mean,bldg: -0.7988023057131248 56.96274977501869 19.330286960578878 0.3393496107004349 Eagle_education_Petra
Building Eagle_lodging_Trina
Train on 8747 samples such as [[[ 0.00e+00]
  [-7.17e+00]
  [-1.77e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.95e+02]
  [ 2.19e+00]]

 [[ 0.00e+00]
  [-7.05e+00]
  [-1.74e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.95e+00]]

 [[ 0.00e+00]
  [-6.96e+00]
  [-1.72e+01]
  [ 0.00e+00]
  [ 0.00e+00]
  [ 1.04e+03]
  [ 1.80e+02]
  [ 1.72e+00]]

 [[ 0.00e+00]
  [-6.90e+00]
  [-1.69e+01]
  [ 0.00e+00]
  [ 0.00e+00]


274/274 [==============================] - 1s 4ms/step - loss: 20219.0828
Epoch 2/5
274/274 [==============================] - 1s 4ms/step - loss: 3129.9622
Epoch 3/5
274/274 [==============================] - 1s 4ms/step - loss: 3106.7210
Epoch 4/5
274/274 [==============================] - 1s 4ms/step - loss: 2774.5286
Epoch 5/5
274/274 [==============================] - 1s 4ms/step - loss: 2917.5134
cor,mean,rmse,rmse/mean,bldg: -0.6796379179294123 148.50737262033823 65.48207992798612 0.44093487597677883 Eagle_education_Teresa
Building Eagle_office_Norbert
Building Eagle_lodging_Casey
Building Eagle_office_Tia
Building Eagle_office_Remedios
Building Eagle_office_Patrice
Building Eagle_education_Shana
History 24 Future 1
Column 1: Correlation of steam and dewTemperature
          Using one weather feature as leading correlate.
Column 2: Mean usage.
          Using mean to help understand the RMSE.
Column 3: RMSE of LinearRegression(X=Weather, y=Usage).
Column 4: RMSE/mean normalized 